# Model evaluation for imbalance data
### Model to test:

- Logistic regression
- VGG16 augmented and group 4 and 5 layer balanced
- VGG16 augmented and group 4 and 5 layer imbalanced 10%
- VGG16 augmented and group 4 and 5 layer imbalanced 20%
- Bayesian model flat
- Bayesian model flat condensed priors
- Bayesian model shrinking


In [48]:
import shutil
#import glob
#from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
import glob
from tqdm import tqdm
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression as CustomLogisticRegression
import plotnine
from plotnine import *

from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array, array_to_img
from keras.models import load_model

os.chdir('/data/kanferg/Images/Pex_project/Transfer_learning/code')
from utils import Taining_data_orgenizer as orgenizer 
from utils import model_builder as mb
from utils import model_evaluation_utils as meu

In [2]:
import os
os.getcwd()

'/gpfs/gsfs10/users/kanferg/Images/Pex_project/Transfer_learning/code'

In [3]:
os.chdir('/data/kanferg/Images/Pex_project/Transfer_learning/code')
from utils import Taining_data_orgenizer as orgenizer 
from utils import model_builder as mb

### Load models

In [12]:
path_model = '/data/kanferg/Images/Pex_project/Transfer_learning/models/imbalance/'
os.chdir(path_model)
os.listdir()

['cnn_transfer_learning_Augmentation_drop_layer_4and5_20p.h5',
 'flat_proir.pkl',
 'cnn_transfer_learning_Augmentation_drop_layer_4and5.h5',
 'cnn_transfer_learning_Augmentation_drop_layer_4and5_10p_cp.h5',
 'shrinking_priors.pkl',
 'flat_proir_condansed.pkl',
 'cnn_transfer_learning_Augmentation_drop_layer_4and5_10p.h5',
 'lr.pkl']

In [13]:
cnn_basic = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5.h5')
cnn_10p = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_10p.h5')
cnn_20p = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_20p.h5')
cnn_10p_ch = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_10p_cp.h5')

2022-05-27 14:59:16.035478: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-27 14:59:16.822555: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14627 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:91:00.0, compute capability: 7.0


In [77]:
path_orig = '/data/kanferg/Images/Pex_project/SIngle_cell_images_training_set/test_data'
os.chdir(path_orig)
test_files = glob.glob('*')

In [78]:
test_files[:5]

['norm_tsvYbYwA20220503_173550exp001_WT_2-3.png',
 'pheno_QAFuKsmb20220503_180624exp001_PEX3KO7_2-4.png',
 'norm_XgBfMqRw20220503_174059exp001_WT_3-3.png',
 'pheno_lDvhUFkD20220503_180813exp001_PEX3KO7_3-1.png',
 'norm_dUUjvAWS20220503_174559exp001_WT_4-3.png']

In [79]:
import random
random.shuffle(test_files)
class_2 = 3
class_1 = 500
count_pheno = 0
count_norm = 0
IMG_DIM = (150, 150)
tests = []
for i in range(len(test_files)):
    if test_files[i].split('/')[0].split('_')[0].strip()=='pheno':
            if count_pheno < class_2:
                tests.append(test_files[i])
                count_pheno += 1
    else:
        if count_norm < class_1:
            tests.append(test_files[i])
            count_norm += 1
test_imgs = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in tests]
test_imgs = np.array(test_imgs)
test_labels = [fn.split('/')[0].split('_')[0].strip() for fn in tests]
test_imgs_scaled = test_imgs.astype('float32')
test_imgs_scaled /= 255
num2class_label_transformer = lambda l: ['norm' if x[0] == 0 else 'pheno' for x in l]
class2num_label_transformer = lambda l: [0 if x == 'pheno' else 1 for x in l]
test_labels_enc = class2num_label_transformer(test_labels[:5])


In [80]:
print('Test dataset shape:{}'.format(test_imgs.shape))
print(test_labels[0:5], test_labels_enc[0:5])

Test dataset shape:(503, 150, 150, 3)
['pheno', 'pheno', 'pheno', 'norm', 'norm'] [0, 0, 0, 1, 1]


In [81]:
tests

['pheno_QlDOujKQ20220503_180547exp001_PEX3KO7_2-4.png',
 'pheno_iLWsDlbL20220503_180627exp001_PEX3KO7_2-4.png',
 'pheno_AkvxcTMT20220503_175513exp001_PEX3KO7_1-2.png',
 'norm_krZDEnIh20220503_173705exp001_WT_2-4.png',
 'norm_xHAAkLGx20220503_175108exp001_WT_5-3.png',
 'norm_LortnpLx20220503_172851exp001_WT_1-2.png',
 'norm_MfstAQiW20220503_175032exp001_WT_5-3.png',
 'norm_hXJJBrAP20220503_172920exp001_WT_1-2.png',
 'norm_IclmAJQT20220503_174539exp001_WT_4-3.png',
 'norm_KRAOKTvN20220503_172743exp001_WT_1-1.png',
 'norm_sKtqPLNv20220503_175202exp001_WT_5-4.png',
 'norm_nqyNYGFW20220503_173008exp001_WT_1-3.png',
 'norm_pzjEiHZG20220503_174926exp001_WT_5-2.png',
 'norm_ZCgbhBae20220503_174332exp001_WT_4-1.png',
 'norm_mYENuKVS20220503_173749exp001_WT_2-4.png',
 'norm_gKVxeAxA20220503_175039exp001_WT_5-3.png',
 'norm_ytJhviOh20220503_172748exp001_WT_1-1.png',
 'norm_DyJMBRrP20220503_174327exp001_WT_4-1.png',
 'norm_yUfeRHxE20220503_172834exp001_WT_1-2.png',
 'norm_KOkRbRZB20220503_175034ex

In [82]:
cnn_basic = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5.h5')
cnn_10p = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_10p.h5')
cnn_20p = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_20p.h5')
cnn_10p_ch = load_model('cnn_transfer_learning_Augmentation_drop_layer_4and5_10p_cp.h5')


OSError: No file or directory found at cnn_transfer_learning_Augmentation_drop_layer_4and5.h5

In [83]:
predictions = cnn_basic.predict(test_imgs_scaled, verbose=0)
classes_x=[np.where(lab >0.5,1,0).tolist() for lab in predictions]
predictions_label = num2class_label_transformer(classes_x)
predictions_label
meu.get_metrics(test_labels, predictions_label)

Accuracy: 0.9861
Precision: 0.9958
Recall: 0.9861
F1 Score: 0.9898


In [84]:
df = pd.DataFrame({'test':test_labels,'pred':predictions.tolist()})

In [86]:
df.loc[df["test"]=='pheno',:]

,test,pred
0,pheno,[0.9999972581863403]
1,pheno,[0.999997615814209]
2,pheno,[0.9999316930770874]


In [89]:
predictions = cnn_10p_ch.predict(test_imgs_scaled, verbose=0)
classes_x=[np.where(lab >0.5,1,0).tolist() for lab in predictions]
predictions_label = num2class_label_transformer(classes_x)
predictions_label
meu.get_metrics(test_labels, predictions_label)

Accuracy: 0.9841
Precision: 0.9957
Recall: 0.9841
F1 Score: 0.9886


In [90]:
df = pd.DataFrame({'test':test_labels,'pred':predictions.tolist()})
df.loc[df["test"]=='pheno',:]

,test,pred
0,pheno,[1.0]
1,pheno,[1.0]
2,pheno,[1.0]


In [91]:
predictions = cnn_10p.predict(test_imgs_scaled, verbose=0)
classes_x=[np.where(lab >0.5,1,0).tolist() for lab in predictions]
predictions_label = num2class_label_transformer(classes_x)
predictions_label
meu.get_metrics(test_labels, predictions_label)

Accuracy: 0.9841
Precision: 0.9957
Recall: 0.9841
F1 Score: 0.9886


In [92]:
df = pd.DataFrame({'test':test_labels,'pred':predictions.tolist()})
df.loc[df["test"]=='pheno',:]

,test,pred
0,pheno,[1.0]
1,pheno,[1.0]
2,pheno,[1.0]


In [93]:
predictions = cnn_20p.predict(test_imgs_scaled, verbose=0)
classes_x=[np.where(lab >0.5,1,0).tolist() for lab in predictions]
predictions_label = num2class_label_transformer(classes_x)
predictions_label
meu.get_metrics(test_labels, predictions_label)

Accuracy: 0.9901
Precision: 0.9963
Recall: 0.9901
F1 Score: 0.9923


In [94]:
df = pd.DataFrame({'test':test_labels,'pred':predictions.tolist()})
df.loc[df["test"]=='pheno',:]

,test,pred
0,pheno,[1.0]
1,pheno,[1.0]
2,pheno,[0.9999998807907104]
